##### Imports

In [21]:
import os
import time
import ee
import requests
import shutil, sys
import logging
import multiprocessing
from retry import retry

### Earth Engine Authentication

In [22]:
#ee.Authenticate()

In [23]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

### Data Loading

In [24]:
def mask_L8_SR (img):
    '''takes in an image and sets a mask for its low quality pixels'''
    qa = img.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3)
    shadow = qa.bitwiseAnd(1 << 4)
    Mask = (cloud.neq(0)).Or(shadow.neq(0)).uint8()
    return  img.updateMask(Mask.eq(0))


#### Year selection

In [65]:
# ---> ENTER Year BELOW <---
YEAR = 2013
print(f'YEAR: {YEAR}')

YEAR: 2013


#### Country selection

In [66]:
# ---> ENTER COUNTRY BELOW <---
COUNTRY = "C�te d'Ivoire"

countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
border = countries.filter(ee.Filter.eq('ADM0_NAME', COUNTRY))
img = ee.Image(1).clip(border)

pointsROI = img.stratifiedSample(numPoints = 1000000,
                                classBand = 'constant',
                                region = border,
                                scale = 10000,
                                geometries = True)

print(f'COUNTRY: {COUNTRY}, POINTS: {pointsROI.size().getInfo()}')

COUNTRY: C�te d'Ivoire, POINTS: 3267


#### Datasets

In [67]:
# Landsat-8 (input)
L8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                .filterDate(f'{YEAR}-01-01',f'{YEAR + 1}-01-01')
                .map(lambda x: mask_L8_SR(x)
                .select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
                        ,['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']))).median() 


In [68]:
# Nightlight (input)
VIIRS = (ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')
                  .filterDate(f'{YEAR}-01-01',f'{YEAR + 1}-01-01')
                  .select(['avg_rad'],['NL'])).median()


#### Data retrieval

In [69]:
# specify output directory
LOCATION = 'abidjan'
OUT_DIR = f'/mimer/NOBACKUP/groups/globalpoverty1/albin_and_albin/{LOCATION}'

# create directory if it does not exist
if not os.path.isdir(OUT_DIR):
        os.makedirs(OUT_DIR, 0o775)
        os.makedirs(OUT_DIR + '/input', 0o775)
        os.makedirs(OUT_DIR + '/output', 0o775)
        print(f'Directory created (mimer): ..{OUT_DIR.partition("globalpoverty1")[-1]}')

In [70]:
@retry(tries=10, delay=1, backoff=2)
def get_image_tile(point):
    '''Retrieve image tile at given point, scale and dimension.  Write to directory'''
    
    # get ID and point-coordinates
    ID = point['id']
    point = ee.Geometry.Point(point['geometry']['coordinates'])
    
    # set up rectangular bound around point
    ROI = point.buffer(500*10).bounds()  # tile dim: 1000*1000px (1px=10m)
    
    # images to retrieve
    imgLandsat = L8.clip(ROI)
    imgNL = VIIRS.clip(ROI)
    
    # concatenate input
    #imageInput = ee.Image.cat([imgLandsat, imgNL, imgSMOD, imgTarget_esa, imgTarget_esri])
    imageInput = ee.Image.cat([imgLandsat, imgNL])

    # fetch the URL from which to download the image.
    url = imageInput.float().getDownloadUrl({
        'scale': 10,
        'dimensions': '1000x1000',
        'format': 'GEO_TIFF'
    })
    r = requests.get(url) # send get request
    
    # save retrieved tile
    if r.status_code == 200:  # HTTP GET: 200 OK
        filename = OUT_DIR + f'/tile_{ID}.tif_{YEAR}'
        with open(filename, 'wb') as out_file:
              out_file.write(r.content)
    # retry, get request failed
    else:
        #print(f'{r.status_code}: {r.reason}')
        raise HTTPException(status_code=r.status_code, detail=r.reason)
    
    return r.content


In [71]:
# make a list of the points
image_points = pointsROI.toList(pointsROI.size().getInfo()).getInfo()

# point indices (found by visual inspection in ee)
im_indices = [2755, 1229, 1991, 1440,
              872, 1252, 3078, 2118,
              224, 2336, 1560, 2303,
              135, 2921, 2072, 1565]

for im_idx in im_indices:    
    
    # retrieve one point at a time
    point_to_download = image_points[im_idx]

    # get corresponding image tiles
    get_image_tile(point_to_download)

    print(f'redownload complete: {COUNTRY}, {YEAR}, tile_{point_to_download["id"]}')

redownload complete: C�te d'Ivoire, 2013, tile_2755
redownload complete: C�te d'Ivoire, 2013, tile_1229
redownload complete: C�te d'Ivoire, 2013, tile_1991
redownload complete: C�te d'Ivoire, 2013, tile_1440
redownload complete: C�te d'Ivoire, 2013, tile_872
redownload complete: C�te d'Ivoire, 2013, tile_1252
redownload complete: C�te d'Ivoire, 2013, tile_3078
redownload complete: C�te d'Ivoire, 2013, tile_2118
redownload complete: C�te d'Ivoire, 2013, tile_224
redownload complete: C�te d'Ivoire, 2013, tile_2336
redownload complete: C�te d'Ivoire, 2013, tile_1560
redownload complete: C�te d'Ivoire, 2013, tile_2303
redownload complete: C�te d'Ivoire, 2013, tile_135
redownload complete: C�te d'Ivoire, 2013, tile_2921
redownload complete: C�te d'Ivoire, 2013, tile_2072
redownload complete: C�te d'Ivoire, 2013, tile_1565
